In [112]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')  # Usar TkAgg backend para ventanas externas


use_gpu = False  # Cambiar a False si no tienes GPU o CuPy instalado
# np} = cp if use_gpu else np

# Consideraciones inciales.

### Funcion para construir la grilla inicial

In [113]:
# Función que construye la grilla inicial

def create_grid(d, grid_size, m, pattern='random'):
    """
    Crea una grilla n-dimensional
    
    Parámetros:
    - d: número de dimensiones
    - grid_size: tamaño de cada dimensión (int o lista de int)
    - m: valor máximo de estado (estados van de 0 a m + 1)
    - pattern: tipo de patrón inicial ('random', 'central', etc.)
    """
    # Convertir grid_size a una tupla si es un solo número
    if isinstance(grid_size, int):
        grid_size = tuple([grid_size] * d)
    else:
        grid_size = tuple(grid_size)
    
    # Crear la grilla según el patrón solicitado
    if pattern == 'random':
        # Grilla con valores aleatorios entre 0 y m
        return np.random.randint(0, m+1, grid_size)
    
    elif pattern == 'central':
        # Grilla con valor alto en el centro y ceros alrededor
        grid = np.zeros(grid_size, dtype=int)
        center = tuple(s // 2 for s in grid_size)
        
        # Colocar valor m en el centro
        indices = tuple(slice(c-1, c+2) for c in center)
        grid[indices] = m
        return grid
    
    elif pattern == 'checkerboard':
        # Patrón de tablero de ajedrez
        indices = np.indices(grid_size)
        sum_indices = np.sum(indices, axis=0)
        return (sum_indices % 2) * m
    
    else:
        # Patrón predeterminado: valores 0
        return np.zeros(grid_size, dtype=int)
    


### Valores inciales

In [125]:
d = 2  # 3 dimensiones de la grilla
grid_size = 10  # Tamaño de los arreglos
m = 2  # Estados posibles por casilla
r = 1 # Radio de vecindad
grid = create_grid(d, grid_size, m, pattern='random') # Grilla inicial

t = 10 # Tiempo


In [115]:
grid

array([[0, 2, 2, 0, 0, 1, 1, 0, 2, 1],
       [0, 1, 1, 0, 0, 2, 1, 1, 0, 1],
       [0, 1, 1, 2, 2, 1, 2, 0, 2, 2],
       [2, 2, 2, 1, 0, 2, 1, 0, 1, 2],
       [2, 0, 1, 1, 2, 0, 0, 1, 2, 1],
       [2, 2, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 1, 2, 0, 0, 0],
       [0, 1, 1, 0, 2, 0, 2, 1, 0, 2],
       [2, 2, 2, 2, 0, 2, 2, 0, 0, 2],
       [1, 0, 2, 0, 0, 0, 1, 0, 0, 1]], dtype=int32)

In [116]:
def evolucionar_grilla(grilla, m, r):
    """
    Evoluciona la grilla de un autómata celular en d dimensiones sin usar bucles.
    
    Parámetros:
    - grilla: ndarray de numpy o cupy con la configuración actual.
    - m: Número máximo de estados por celda.
    - r: Radio de vecindad.
    
    Retorna:
    - Nueva grilla evolucionada según las reglas del autómata.
    """
    # Expandir la grilla en un toroide (bordes periódicos)
    grilla_expandida = np.pad(grilla, r, mode='wrap')
    
    # Construir el kernel para calcular la suma de vecinos
    kernel_shape = (2 * r + 1,) * grilla.ndim  # Vecindad cúbica de radio r
    kernel = np.ones(kernel_shape, dtype=grilla.dtype)
    
    # Excluir la celda central
    indices_centro = tuple(slice(r, r+1) for _ in range(grilla.ndim))
    kernel[indices_centro] = 0
    
    # Verificar la suma total del kernel (debe ser el número de vecinos)
    # print("Suma del kernel:", np.sum(kernel))
    
    # Convolución para obtener la suma de vecinos
    from scipy.ndimage import convolve
    suma_vecinos = convolve(grilla_expandida, kernel, mode='constant', cval=0)
    
    # Extraer la parte válida de la convolución
    slices = tuple(slice(r, -r) for _ in range(grilla.ndim))
    suma_vecinos = suma_vecinos[slices]
    
    # Verificar si las sumas de vecinos son razonables
    # print("Sumas de vecinos:", suma_vecinos)
    
    # Calcular el valor máximo de suma correctamente
    # Para una grilla 2D con r=1, debería ser m × 3 (para una grilla 2×2)
    num_vecinos = np.sum(kernel)
    SM = m * num_vecinos
    intervalos = np.linspace(0, SM, 4)  # Dividimos en 3 intervalos
    
    # Aplicar las reglas de evolución
    nueva_grilla = np.copy(grilla)
    nueva_grilla[(suma_vecinos >= intervalos[0]) & (suma_vecinos < intervalos[1])] -= 1
    nueva_grilla[(suma_vecinos >= intervalos[1]) & (suma_vecinos < intervalos[2])] += 1
    nueva_grilla[(suma_vecinos >= intervalos[2])] -= 1
    
    return np.clip(nueva_grilla, 0, m)  # Asegurar valores en [0, m]

In [117]:
grilla_evolucionada = evolucionar_grilla(grid, m, r)
# i = 1
# for i in range(100):
    



# print(f"grilla original: \n", grid)

# print(f"grilla final: \n", grilla_evolucionada)

# # plt.show()

In [118]:
# 3d

grilla_3d = np.array([
    # Capa 0 (z=0)
    [[1, 2],
     [0, 1]],
    
    # Capa 1 (z=1)
    [[2, 0],
     [1, 2]]
])


m = 3  # Valor máximo de estado
r = 1  # Radio de vecindad

# Evolucionar la grilla 3D
grilla_evolucionada = evolucionar_grilla(grilla_3d, m, r)

print("Grilla original:")
print(grilla_3d)
print("\nGrilla evolucionada:")
print(grilla_evolucionada)

Grilla original:
[[[1 2]
  [0 1]]

 [[2 0]
  [1 2]]]

Grilla evolucionada:
[[[2 3]
  [1 2]]

 [[3 0]
  [2 3]]]


In [119]:



def visualizar_grilla_2d(grilla, titulo="Grilla del Autómata Celular", cmap='viridis'):
    """
    Visualiza una grilla 2D con mapa de colores.
    
    Parámetros:
    - grilla: Array 2D de numpy/cupy
    - titulo: Título para la figura
    - cmap: Mapa de colores (viridis, jet, plasma, inferno, magma, etc.)
    """
    # Si estamos usando cupy, convertir a numpy
    if hasattr(grilla, 'get'):
        grilla = grilla.get()
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Crear la imagen coloreada
    im = ax.imshow(grilla, cmap=cmap, interpolation='nearest')
    
    # Agregar barra de colores
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Estado de la celda')
    
    # Configurar rejilla y ticks
    ax.set_xticks(np.arange(-.5, grilla.shape[1], 1), minor=True)
    ax.set_yticks(np.arange(-.5, grilla.shape[0], 1), minor=True)
    ax.grid(which='minor', color='w', linestyle='-', linewidth=1)
    
    # Añadir valores en cada celda
    for i in range(grilla.shape[0]):
        for j in range(grilla.shape[1]):
            ax.text(j, i, str(grilla[i, j]), 
                    ha="center", va="center", color="w" if grilla[i, j] > grilla.max()/2 else "black")
    
    plt.title(titulo)
    plt.tight_layout()
    plt.show()

In [122]:
visualizar_grilla_2d(grid)

In [121]:
visualizar_grilla_2d(grilla_evolucionada)

TypeError: Invalid shape (2, 2, 2) for image data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time

def simular_con_visualizacion_2d(grilla_inicial, m, r, iteraciones=10, pausa=1, cmap='viridis'):
    """
    Simula la evolución del autómata celular 2D y muestra las imágenes a medida 
    que avanzan las iteraciones.
    
    Parámetros:
    - grilla_inicial: Grilla inicial 2D
    - m: Valor máximo de estado
    - r: Radio de vecindad
    - iteraciones: Número de iteraciones a simular
    - pausa: Tiempo entre iteraciones (segundos)
    - cmap: Mapa de colores
    """
    # Iniciar simulación
    grilla_actual = grilla_inicial.copy()
    
    # Mostrar grilla inicial
    plt.figure(figsize=(8, 6))
    im = plt.imshow(grilla_actual, cmap=cmap, interpolation='nearest')
    plt.colorbar(label='Estado de la celda')
    plt.title(f"Paso 0 (Inicial)")
    
    # Configurar rejilla
    plt.grid(True, which='both', color='white', linestyle='-', alpha=0.3)
    plt.xticks(np.arange(-.5, grilla_actual.shape[1], 1), [])
    plt.yticks(np.arange(-.5, grilla_actual.shape[0], 1), [])
    
    # Agregar valores en cada celda
    for i in range(grilla_actual.shape[0]):
        for j in range(grilla_actual.shape[1]):
            plt.text(j, i, str(grilla_actual[i, j]), 
                    ha="center", va="center", 
                    color="w" if grilla_actual[i, j] > m/2 else "black")
    
    plt.show()
    
    # Simular y mostrar cada iteración
    for paso in range(1, iteraciones + 1):
        # Evolucionar la grilla
        grilla_actual = evolucionar_grilla(grilla_actual, m, r)
        
        # Crear nueva figura para este paso
        plt.figure(figsize=(8, 6))
        im = plt.imshow(grilla_actual, cmap=cmap, interpolation='nearest')
        plt.colorbar(label='Estado de la celda')
        plt.title(f"Paso {paso}")
        
        # Configurar rejilla
        plt.grid(True, which='both', color='white', linestyle='-', alpha=0.3)
        plt.xticks(np.arange(-.5, grilla_actual.shape[1], 1), [])
        plt.yticks(np.arange(-.5, grilla_actual.shape[0], 1), [])
        
        # Agregar valores en cada celda
        for i in range(grilla_actual.shape[0]):
            for j in range(grilla_actual.shape[1]):
                plt.text(j, i, str(grilla_actual[i, j]), 
                        ha="center", va="center", 
                        color="w" if grilla_actual[i, j] > m/2 else "black")
        
        plt.show()
        time.sleep(pausa)  # Pausa entre iteraciones

In [ ]:

# Simular 5 iteraciones, con 1 segundo entre cada una
simular_con_visualizacion_2d(grid, m, r, iteraciones=5, pausa=1)

In [ ]:


def visualizar_automata_secuencial(grilla_inicial, m, r, iteraciones=5, pausa=1, cmap='viridis'):
    """
    Simula y visualiza la evolución del autómata celular en ventanas secuenciales.
    Al cerrar una ventana, aparecerá automáticamente la siguiente.
    
    Parámetros:
    - grilla_inicial: Grilla inicial 2D
    - m: Valor máximo de estado
    - r: Radio de vecindad
    - iteraciones: Número de iteraciones a simular
    - pausa: Tiempo entre iteraciones (segundos)
    - cmap: Mapa de colores
    """
    # Desactivar modo interactivo
    plt.ioff()
    
    # Iniciar simulación
    grilla_actual = grilla_inicial.copy()
    
    # Lista para almacenar todas las grillas generadas
    historial = [grilla_actual.copy()]
    
    # Generar todas las iteraciones primero
    for paso in range(iteraciones):
        grilla_actual = evolucionar_grilla(grilla_actual, m, r)
        historial.append(grilla_actual.copy())
    
    # Mostrar cada grilla secuencialmente
    for paso, grilla in enumerate(historial):
        fig = plt.figure(figsize=(8, 6), num=f"Autómata Celular - Paso {paso}")
        plt.imshow(grilla, cmap=cmap, interpolation='nearest')
        plt.colorbar(label='Estado de la celda')
        plt.title(f"Paso {paso}" + (" (Inicial)" if paso == 0 else ""))
        plt.grid(True, which='both', color='white', linestyle='-', alpha=0.3)
        
        # Agregar valores en cada celda
        for i in range(grilla.shape[0]):
            for j in range(grilla.shape[1]):
                plt.text(j, i, str(grilla[i, j]), 
                       ha="center", va="center", 
                       color="w" if grilla[i, j] > m/2 else "black")
        
        # Mostrar con bloqueo - espera hasta que la ventana se cierre
        if paso < len(historial) - 1:
            plt.tight_layout()
            plt.show(block=True)
            plt.close()
        else:
            # Para la última ventana, mostrar sin bloqueo
            plt.tight_layout()
            plt.show()
    
    return historial[-1]  # Retornar estado final

In [ ]:
estado_final = visualizar_automata_secuencial(grid, m, r, iteraciones=100)